# Interval Around the Current Price
This strategy will provide liquidity around the current price in a fixed interval. We will rebalance our position every update interval as discussed in the beginning of chapter 4. Let pc be the current price and a ∈ [10, 40000], then we will set our position to [pc − a, pc + a] at every update interval.
We will fill the interval as much as possible. However, it is usually not possible to use up both tokens, so the remaining token will be left outside the pool.

In [ ]:
from datetime import timedelta, date

import pandas as pd

from demeter import TokenInfo, Actuator, ChainType, MarketInfo, Strategy
from demeter.uniswap import UniV3Pool, UniLpMarket

In [ ]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

In [ ]:
class IntervalsAroundtheCurrentPrice(Strategy):
    def __init__(self, a=10, b=1, update_interval=timedelta(days=1)):
        super().__init__()
        self.a = a
        self.b = b

    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        init_price = lp_market.market_status.data.price

        lp_market.add_liquidity(init_price - self.a, init_price + self.a)

In [ ]:
usdc = TokenInfo(name="usdc", decimal=6)  # declare  token0
eth = TokenInfo(name="eth", decimal=18)  # declare token1
pool = UniV3Pool(usdc, eth, 0.05, usdc)  # declare pool
market_key = MarketInfo("uni_market")

actuator = Actuator()  # declare actuator
broker = actuator.broker
market = UniLpMarket(market_key, pool)

broker.add_market(market)
broker.set_balance(usdc, 2000)
broker.set_balance(eth, 0)

actuator.strategy = IntervalsAroundtheCurrentPrice(400, 200)

market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 13), date(2023, 8, 17))
actuator.set_price(market.get_price_from_data())
actuator.run()